In [42]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'bigquery.json'
os.environ['http_proxy'] = "" 
os.environ['https_proxy'] = ""
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file('model_team_bigquery.json')
import pandas_gbq
query_str = """
SELECT * FROM `xed-project-237404.footprint_model_etl.address_balance_bin` 
-- where balance_bin = '[20 - 200]'  or balance_bin = '[200 - 1000]'  or balance_bin = '[1000 - 10000]'  
where balance_bin != '[0 - 20]' and balance_bin != 'NA'
"""
project_id = 'xed-project-237404'
df = pandas_gbq.read_gbq(query_str, project_id=project_id) #, credentials=credentials)


Downloading: 100%|██████████| 59274629/59274629 [2:33:16<00:00, 6445.24rows/s]


In [43]:
df.to_csv('address_balance_bin.csv')

In [ ]:
df = pd.read_csv('address_balance_bin.csv')

In [45]:
df.head(3)

,address,eth_balance_normal,balance_bin
0,0x7c9265d1af3bd9bd4ec4f21228618d21b5faf261,2.000000e-08,(0 - 20]
1,0x1949d72f1196bf02c669b96135270546a0d288b4,2.137460e-03,(0 - 20]
2,0xdfc3e63d82ce6179b121eb36088f47e769b5d5c8,6.000000e-10,(0 - 20]


In [46]:
import pandas as pd
df_rel=pd.read_csv('bq-results-20210801-202031-eh5uffrutzh7.csv')

In [47]:
df_rel.head(3)

,from_address,to_address,min_date,max_date,cnt_transaction,sum_transaction
0,0x7fe422272db89fa29473026a979051e5c5d8fcf2,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,2021-04-09,2021-04-25,10,710000000000000000
1,0x51102afca1fb28c349359643e2a28d8e40b97b8b,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,2021-04-16,2021-05-04,15,39000000000000000000
2,0x3ecef08d0e2dad803847e052249bb4f8bff2d5bb,0x937409762841bf78cac45e87a7773f905cbc7b9d,2021-04-02,2021-06-05,21,1217332510000000000


In [48]:
len(df_rel[df_rel.cnt_transaction>10])

636366

In [49]:
df_found=df_rel[df_rel.cnt_transaction>10].merge(df,   left_on='from_address', right_on='address')
df_found.rename(columns={'balance_bin':'balance_bin_from'}, inplace=True)
df_found.drop(columns=['eth_balance_normal','address'] , inplace=True)
df_found

,from_address,to_address,min_date,max_date,cnt_transaction,sum_transaction,balance_bin_from
0,0x51102afca1fb28c349359643e2a28d8e40b97b8b,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,2021-04-16,2021-05-04,15,39000000000000000000,(0 - 20]
1,0x3ecef08d0e2dad803847e052249bb4f8bff2d5bb,0x937409762841bf78cac45e87a7773f905cbc7b9d,2021-04-02,2021-06-05,21,1217332510000000000,[1000 - 10000]
2,0x3ecef08d0e2dad803847e052249bb4f8bff2d5bb,0x9b3002bbd6c1d09d63d6e5fe7d393fd5dd95e796,2021-04-04,2021-05-04,11,1301366510000000000,[1000 - 10000]
3,0x3ecef08d0e2dad803847e052249bb4f8bff2d5bb,0xac7a076ae61a93e77db0cddb9f01dee0631970d4,2021-04-04,2021-05-04,18,2591755280000000000,[1000 - 10000]
4,0x3ecef08d0e2dad803847e052249bb4f8bff2d5bb,0x7328db9e0a97887ca1a862ab4e9f248b53d4d480,2021-04-03,2021-04-21,15,125126430000000000,[1000 - 10000]
...,...,...,...,...,...,...,...
533241,0x7bc2031cc88e61fd293a70e3c15edc0d8d41a846,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,2021-05-18,2021-08-01,11,6608000000000000000,(0 - 20]
533242,0xc301625d64528e69ddf101914188fcd793c528cc,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,2021-07-08,2021-08-01,13,602300000000000000,(0 - 20]
533243,0xcb6538adf5d42b36ca7e9aa34f5f98f22cba312c,0x03f7724180aa6b939894b5ca4314783b0b36b329,2021-07-13,2021-08-01,13,98930472038673641,(0 - 20]
533244,0xeb0c1af4dc66db79ea67e0c4fe93362e418695a2,0x881d40237659c251811cec9c364ef91dc08d300c,2021-04-03,2021-08-01,13,3334500000000000000,(0 - 20]


In [50]:
df_rel_fin=df_found.merge(df, left_on='to_address', right_on='address'  )
df_rel_fin.rename(columns={'balance_bin':'balance_bin_to'}, inplace=True)
df_rel_fin.drop(columns=['eth_balance_normal','address'], inplace=True)
df_rel_fin


,from_address,to_address,min_date,max_date,cnt_transaction,sum_transaction,balance_bin_from,balance_bin_to
0,0x3ecef08d0e2dad803847e052249bb4f8bff2d5bb,0x9b3002bbd6c1d09d63d6e5fe7d393fd5dd95e796,2021-04-04,2021-05-04,11,1301366510000000000,[1000 - 10000],(0 - 20]
1,0x3ecef08d0e2dad803847e052249bb4f8bff2d5bb,0x7328db9e0a97887ca1a862ab4e9f248b53d4d480,2021-04-03,2021-04-21,15,125126430000000000,[1000 - 10000],(0 - 20]
2,0x3ecef08d0e2dad803847e052249bb4f8bff2d5bb,0x6511b9d582389419f85b039651924d84efe7b9ab,2021-04-02,2021-05-05,35,28695268000000000000,[1000 - 10000],(0 - 20]
3,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0x6511b9d582389419f85b039651924d84efe7b9ab,2021-05-05,2021-07-27,43,28287488290738935836,[1000 - 10000],(0 - 20]
4,0x49b21bdfa30333858956342f4028ce72e37eb851,0x6511b9d582389419f85b039651924d84efe7b9ab,2021-06-17,2021-08-01,46,30167730000000000000,[1000 - 10000],(0 - 20]
...,...,...,...,...,...,...,...,...
264062,0xd30eba877be3dfbc44bfa06fa1cbece0f04e4b7a,0x57c427b7dad737dec9d668d270e231b4ff469e24,2021-05-18,2021-08-01,16,32130000000000000000,(0 - 20],(0 - 20]
264063,0xf39ee1e9208ab84f5d62b192bcc774b331de6ded,0xf0d2d9baae82e07faad9d76f9ce5b16ed9d6020d,2021-05-16,2021-08-01,11,4385900000000000000,(0 - 20],(0 - 20]
264064,0x29ff371ed163df22ea2110847b7ad26bbb6a719c,0x72e9e8c21936f868cbb9e52a4939cac9cf92c70f,2021-04-29,2021-08-01,25,3283741562841708034,(0 - 20],(0 - 20]
264065,0x0c827a9651630be28444d9f6ee21d997f2f3d272,0x5528d82423d91da8e8fe8066fab15cc014ebd5e2,2021-04-05,2021-08-01,19,3469976872263289724958,[200 - 1000],[20 - 200]


In [51]:
df_rel_fin.to_csv('rel.csv')

In [52]:
df_rel_fin[df_rel_fin.balance_bin_from != df_rel_fin.balance_bin_to]

,from_address,to_address,min_date,max_date,cnt_transaction,sum_transaction,balance_bin_from,balance_bin_to
0,0x3ecef08d0e2dad803847e052249bb4f8bff2d5bb,0x9b3002bbd6c1d09d63d6e5fe7d393fd5dd95e796,2021-04-04,2021-05-04,11,1301366510000000000,[1000 - 10000],(0 - 20]
1,0x3ecef08d0e2dad803847e052249bb4f8bff2d5bb,0x7328db9e0a97887ca1a862ab4e9f248b53d4d480,2021-04-03,2021-04-21,15,125126430000000000,[1000 - 10000],(0 - 20]
2,0x3ecef08d0e2dad803847e052249bb4f8bff2d5bb,0x6511b9d582389419f85b039651924d84efe7b9ab,2021-04-02,2021-05-05,35,28695268000000000000,[1000 - 10000],(0 - 20]
3,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0x6511b9d582389419f85b039651924d84efe7b9ab,2021-05-05,2021-07-27,43,28287488290738935836,[1000 - 10000],(0 - 20]
4,0x49b21bdfa30333858956342f4028ce72e37eb851,0x6511b9d582389419f85b039651924d84efe7b9ab,2021-06-17,2021-08-01,46,30167730000000000000,[1000 - 10000],(0 - 20]
...,...,...,...,...,...,...,...,...
264019,0x7af76636450c683ed73fde92e661a48c7704deb6,0xa10e2eb71a007c3ec793f5335df939937412aafa,2021-04-15,2021-08-01,14,32500000000000000000,[20 - 200],(0 - 20]
264028,0x9b3c1b52d59293ca120b14e744ef58b74d53dfc3,0xf7aab787787631d5d180b54b83747e6654e8f6b6,2021-04-14,2021-08-01,11,264629732386924728486,(0 - 20],[20 - 200]
264035,0x2a860183daea77dba6aef2ab69364f5c18f63c61,0x3d9388892c72a27db307aa8f4512c1057de88434,2021-04-09,2021-08-01,12,1354790000000000000000,[200 - 1000],(0 - 20]
264045,0xe51890210106c3e82ccedf14d39cc546defb9856,0xff58ad2ce6471d44ede0d7458340c2043cbc7cb2,2021-07-11,2021-08-01,20,234550000000000000000,(0 - 20],[20 - 200]


In [ ]:
# not on

In [58]:
df_cross=df_rel_fin[df_rel_fin.balance_bin_from != df_rel_fin.balance_bin_to]
df_cross

In [61]:
df_cross2=df_rel_fin.groupby(['balance_bin_from','balance_bin_to'])['sum_transaction'].agg(['sum','count'])
df_cross2

sum  \
balance_bin_from balance_bin_to                                                        
(0 - 20]         (0 - 20]          3910520677105515652106275093295974104155154379...   
                 [1000 - 10000]    1919058575077007202725435661000000000000714938...   
                 [10000 - 20000]   1025413150000000000000055124170000000000000007...   
                 [100000+]         1798155451118421877906321854040000000000000000...   
                 [20 - 200]        3271210317052580707946521670000000000002937310...   
                 [200 - 1000]      3211344207576898131311526266349180640921988483...   
                 [20000 - 100000]  6388023700000000000000177052628950000000007883...   
[1000 - 10000]   (0 - 20]          1301366510000000000125126430000000000286952680...   
                 [1000 - 10000]    2184864082800000000002461696890588690120582555...   
                 [10000 - 20000]   2756198876130000000000426914059153265486104065...   
                 [100000+]         3706171429900553221032136100500000000000000000...   
                 [20 - 200]        2110839980000000000481919355212474700870864908...   
                 [200 - 1000]      1701747228500000000004138726083000000000097655...   
                 [20000 - 100000]  5333226638254489916532405194500000000002376580...   
[10000 - 20000]  (0 - 20]          7812120900000000008998697800000000004995663160...   
                 [1000 - 10000]    3798828854000000000000502985182200000000000059...   
                 [10000 - 20000]   2197127948300000000000020792887000000000000000...   
                 [20 - 200]        5590000000000000000007658500000000000000001169...   
                 [200 - 1000]      1219310000000000000000144094185271500000000001...   
                 [20000 - 100000]  8004000000000000000000189750000000000000000002...   
[100000+]        (0 - 20]                                   259507147117260000000000   
                 [20 - 200]                                   2240000000000000000000   
                 [200 - 1000]       473279977157530000000000129001135000000000000000   
[20 - 200]       (0 - 20]          5774000000000000000020619500000000000000270030...   
                 [1000 - 10000]    5251752140000000000001502601448700000000003900...   
                 [10000 - 20000]   3396537000000000000000678700000000000000000608...   
                 [100000+]              19238962191378822880001534335115449908062000   
                 [20 - 200]        1117626512100000000004955608020000000000442093...   
                 [200 - 1000]      8207770009482245110919808346609000000000013018...   
                 [20000 - 100000]      226285600000000000000526977373055000000000000   
[200 - 1000]     (0 - 20]          5534172428512790700144750363383000000000585460...   
                 [1000 - 10000]    2436800000000000000007425753026000000000057898...   
                 [10000 - 20000]   5845230000000000000000243840150000000000000037...   
                 [100000+]           70127445405176536281000445531007970190000000000   
                 [20 - 200]        2836498996600000000248621840317000000001043549...   
                 [200 - 1000]      1226095133999999992321457637493900000000857347...   
                 [20000 - 100000]  5541491196407100020500139747591164118506596152...   
[20000 - 100000] (0 - 20]          2099399000000000000000217526700000000000000022...   
                 [1000 - 10000]    1192000000000000000000296500000000000000000000...   
                 [10000 - 20000]   1862479000000000000000011894501100000000000000...   
                 [100000+]                                  787557000000000000000000   
                 [20 - 200]        1090505000000000000000934337000000000000000385...   
                 [200 - 1000]      4270520000000000000000341292500000000000000065...   

                                    count  
balance_bin_from balance_bin_to            
(0 - 20]         (0 - 20]       

In [63]:
df_cross2.reset_index()

,balance_bin_from,balance_bin_to,sum,count
0,(0 - 20],(0 - 20],3910520677105515652106275093295974104155154379...,32549
1,(0 - 20],[1000 - 10000],1919058575077007202725435661000000000000714938...,9776
2,(0 - 20],[10000 - 20000],1025413150000000000000055124170000000000000007...,134
3,(0 - 20],[100000+],1798155451118421877906321854040000000000000000...,4
4,(0 - 20],[20 - 200],3271210317052580707946521670000000000002937310...,2277
5,(0 - 20],[200 - 1000],3211344207576898131311526266349180640921988483...,3444
6,(0 - 20],[20000 - 100000],6388023700000000000000177052628950000000007883...,1642
7,[1000 - 10000],(0 - 20],1301366510000000000125126430000000000286952680...,183621
8,[1000 - 10000],[1000 - 10000],2184864082800000000002461696890588690120582555...,107
9,[1000 - 10000],[10000 - 20000],2756198876130000000000426914059153265486104065...,26


In [68]:
df_cross2.to_csv('cross_relationship.csv')